<a href="https://colab.research.google.com/github/jyeongvv/DL_study/blob/main/jyeongvv/0405_RNN_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC_%EB%B3%B5%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RNN

* 시퀀스 모델
* RNN은 동영상, 자연어, 주가 등 **동적**인 데이터를 이용할 때 성능이 극대화됨
* **은닉상태(hidden State)**
* **재귀적** 구조
* 맥락을 인식할 수 있는 신경망
* 기울기 소실, 폭발
* GRU, 게이트, 업데이트 게이트

단점
> * 단순히 가중치를 반복 사용해 데이터로부터 순서 정보를 추출
> * 기울기를 역전파할 때, 1보다 커지면 너무 커지고 1보다 작아지면 0에 수렴
> * 시계열 길이가 길어질수록 적용하기 어려움

## 데이터처리

* 시간 없애고 날짜만 남기기
* 시간을 없애기 위해서 -> datetime -> dt.date 
* dtype : object -> 문자열(바꿔주기)
* between(A, B) A와 B 사이의 날짜만 표시 (포함)
* 인덱스를 리셋하고 기존 인덱스를 삭제 -> `reset_index(drop=True)`

## 데이터셋

* `self.data = self.data / np.max(self.data)` -> np.max 해당 배열에서 가장 큰값 -> 0~1 사이로 데이터를 스케일링

## 모델

* RNN 층 정의
  * Open, High, Low
  * `num_layers` -> RNN층을 몇 개 쌓을까
  > * 너무 많으면 -> **기울기 소실** -> 특징을 업데이트 X
  > * 너무 적으면 -> **기울기 폭발** -> 학습 X

* x -> 층을 거쳐 나온 결과 (출력층), hn은 모든 RNN층의 은닉 상태 (누적 가중치)


* FC / MLP => 분류 예측
  * FC(fully-connected, 전결합층)
  * MLP(multi-layer perceptron, 다층 퍼셉트론)층
* x = torch.flatten(x) # 예측한 종가를 1차원 표현
* out_features -> 회귀문제는 1개 분류는 분류하는 클래스 갯수대로       

---

* `optim.zero_grad()` -> 초기 은닉 상태
* RNN 레이어 수, 배치 크기, 은닉층의 면(데이터 들어오는 창구) -> `h0 = torch.zeros(5, data.shape[0], 8).to(device)`
* 사용가능한 배치 개수를 반환하는 __len__()

# 자연어처리

* 자연어 : 일상적으로 사용하는 말을 프로그래밍 언어와 구분하여 부르는 말
* 워드 임베딩 word embedding: 언어의 최소 단위 -> **토큰화** => 벡터화 (라벨 인코딩)
자연어(텍스트) 전처리
* 형태소분석(기) - Mecab, Khaill, Okt

## 자연어 전처리

* 하이퍼 패러미터 정의
* 데이터로딩
* 훈련셋 / 데이터셋
  * label -> 정답
---
* 단어 사전
> * `TEXT.build_vocab(trainset, min_freq=n)` -> n번 이상 등장한 단어로 단어 사전 만들기
> * `LABEL.build_vocab(trainset)` -> 학습데이터에서 5번 미만 등장한 데이터는 unk(unknown)으로 대체


## RNN 모델 구현

> * `n_layers` 층 -> 은닉 벡터들의 층의 갯수 : 엄청 복잡한 모델이 아니라면, n_layers는 2 이하 통상 정의
> * `hidden_dim` 층 내부의 너비
> * `n_vocab` 단어 임베딩 관련
> * `embed_dim` 단어 임베딩 관련
> * `n_classes` 최종적으로 분류할 텍스트의 가짓수 (neg, pos)

---
* 단어 임베딩 -> 단어들을 벡터화(라벨링)
* `n_vocab` : 전체 데이터셋의 모든 단어를 사전 형태로 나타냈을 때, 그 사전에 등재된 단어 수
* `embed_dim` : 임베딩 -> 단어 텐서가 가지는 차원값
* RNN을 통해 생성되는 은닉 벡터의 차원값과 드롭아웃을 정의
```
self.hidden_dim = hidden_dim
self.dropout = nn.Dropout(dropout_p)
```

* 시계열(순서) 데이터 -> 하나의 텐서 압축 `self.out = nn.Linear(hidden_dim, n_classes)`

* `y.data.sub_(1)` -> y값에 일괄적으로 1를 빼줘서, 1과 2를 0과 1로 변환
* x를 모델에 입력해서 예측값인 logit 계산
  * `logit = model(x)` -> logit = 0과 1 사이의 확률
  * `loss = F.cross_entropy(logit, y)` -> 예측값과 정답값(라벨) 비교 -> 손실함수
---
* tensor.max(0) = 각 열에서의 최대값
* tensor.max(1) = 각 행에서의 최대값 / [1] -> 최대값의 인덱스 (0,1)
* max의 결과값 -> (최대값[0], 그 값의 인덱스(indices)[1])

---

* 모델 객체 정의
  * 1 : 내부 GRU의 층 개수
  * 256 : 모델 내 은닉 벡터의 차원값
  * 128 : 임베딩(벡터화)된 토큰의 차원값
  * 0.5 : 드롭아웃 비중
  > model = BasicGRU(1, 256, vocab_size, 128, n_classes, 0.5).to(DEVICE)

#### 학습
* not best_val_loss = None은 not으로 취급 = 아직 최적 검증 오차가 비어있거나..
* val_loss < best_val_loss = 새롭게 구한 검증 오차가 기존에 최적 검증 오차보다 작으면 -> 새로운 모델의 오차가 더 작으면